[Reference](https://medium.com/@kasperjuunge/how-to-use-celery-c34310e6bcba)

# Creating Tasks

In [1]:
from celery import Celery

app = Celery('my_tasks', broker='pyamqp://guest@localhost//')
@app.task
def add(x, y):
    return x + y

# Running Tasks

In [2]:
result = add.delay(4, 6)

# Checking Task States

In [4]:
if result.ready():
    print('Task result: ', result.get(timeout=1))

# Retry Failed Tasks

In [5]:
@app.task(bind=True, max_retries=3)
def data_fetcher(self, url):
    try:
        response = requests.get(url)
        return response.status_code
    except Exception as e:
        raise self.retry(exc=e, countdown=60)

# Periodic Tasks

In [6]:
app.conf.beat_schedule = {
    'add-every-30-seconds': {
        'task': 'tasks.add',
        'schedule': 30.0,
        'args': (16, 16)
    },
}

In [7]:
from celery.schedules import crontab

app.conf.beat_schedule = {
    'add-every-monday-morning': {
        'task': 'tasks.add',
        'schedule': crontab(hour=7, minute=30, day_of_week=1),
        'args': (16, 16),
    },
}

# Task Routing

In [8]:
@app.task(queue='io_intensive')
def io_task():
    pass

@app.task(queue='cpu_intensive')
def cpu_task():
    pass

# Task Chaining

In [9]:
from celery import chain

chain(add.s(4, 4), add.s(5), add.s(6))()

# Task Grouping

In [10]:
from celery import group

job = group(add.s(i, i) for i in range(10))
result = job.apply_async()

# Rate Limiting

In [11]:
@app.task(rate_limit='100/m')  # Allow 100 tasks per minute
def my_task():
    pass

# Task Inheritance

In [12]:
class BaseTask(app.Task):
    def on_failure(self, exc, task_id, args, kwargs, einfo):
        print(f'Task {task_id} failed: {exc}')

@app.task(base=BaseTask)
def add(x, y):
    return x + y

# Task Results

In [13]:
@app.task(ignore_result=True)  # Ignore results, don't store them
def add(x, y):
    return x + y

# Exception Handling

In [14]:
@app.task(throws=(MyCustomException,))
def task_that_might_fail():
    if something_bad:
        raise MyCustomException('Something bad happened')

```
celery -A your_project_name worker --loglevel=info
```